# 线性回归 计算 beta

使用线性回归模型计算出基金的 beta 值，以此作为基金的个分段持仓比例，并且有以下改进：
1. 使用逐步回归法筛选关键期限子段，降低共线性影响。**在逐步回归法筛选后，回补3年以内的短久期子段**。
2. 设置杠杆上限为140%，下限为80%。
3. 在40个交易日的滚动窗口基础上，增加最新一周数据的权重。
4. 根据基金持仓风格（利率、信用、均衡），匹配相应的指数。

总在收益率分别包含 总财富、总净价、总全价三个值，我们应该使用总财富收益率作为指数收益率

# 首先排除基金持仓风格，计算基金的 beta 值

- 利率债基：主要投资于利率债（如国债、政策性金融债、同业存单等），利率持仓超过70%。使用中债国开债指数族和存单指数族，因为这些指数更能反映利率债的收益特征。
- 信用债基：主要投资于信用债（如企业债、中票、短融、非政策性金融债等），信用持仓超过70%。使用二永指数族（二级资本债和永续债）和信用债指数族，因为这些指数更能反映信用债的收益特征。

由于没有开债指数族、存单指数族、二永指数族、信用债指数族的数据，我们使用全部基金的数据和简单的中债分段数据来计算 beta 值



# 计算二次凸优化

我们的目标是找到一个线性组合，使得基金的回报率与中债分段指数的回报率之间的误差最小。

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm


def calc_beta(X, y):
    # 定义变量
    n_features = X.shape[1]
    coefficients = cp.Variable(n_features)

    # 目标函数：最小二乘
    objective = cp.Minimize(cp.sum_squares(X @ coefficients - y))

    # 约束条件
    constraints = [
        coefficients >= 0,
        cp.sum(coefficients) <= 1.4,
        cp.sum(coefficients) >= 0.8,
    ]

    # 求解问题
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # 返回系数值
    if problem.status == 'optimal':
        return coefficients.value
    else:
        return np.nan * np.ones(n_features)  # 如果无解，返回 NaN

In [2]:
def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


In [3]:
from tqdm import tqdm

# 文件路径
input_dist = 'data/combined/'
output_dist = 'data/eda_beta/'

print("reading data, loading into cache")
cache = {}
for file_name in tqdm(os.listdir(input_dist)):
    df = pd.read_csv(input_dist + file_name)
    cache[file_name] = df

if not os.path.exists(output_dist):
    os.makedirs(output_dist)


def process(file_name):
    # 读取数据
    df = cache[file_name]
    df = df.dropna()  # 去除空值

    df = df.copy()

    if df.empty:
        print(f"empty dataframe: {file_name}")
        return  # 空数据直接返回

    # 初始化 beta 列
    beta_columns = ['beta_1年以下', 'beta_1-3年', 'beta_3-5年', 'beta_5-7年', 'beta_7-10年', 'beta_10年以上']
    for col in beta_columns:
        df.loc[:, col] = np.nan  # 初始化为 NaN

    # 滚动计算 beta
    for i in range(40, df.shape[0]):
        X = df[['1年以下', '1-3年', '3-5年', '5-7年', '7-10年', '10年以上']].values[i - 40:i]
        y = df['回报'].values[i - 40:i]
        date = df['日期'].values[i]

        # 把最后五条数据复制三次添加到最后
        X = np.vstack([X, [X[-1]] * 3])  # 使用 np.vstack 保持二维结构
        y = np.append(y, [y[-5]] * 3)

        # 转换为 DataFrame 以便进行逐步回归
        X_df = pd.DataFrame(X, columns=['1年以下', '1-3年', '3-5年', '5-7年', '7-10年', '10年以上'])
        y_series = pd.Series(y)

        # 逐步回归选择变量
        selected_variables = stepwise_selection(X_df, y_series, verbose=False)
        if '1年以下' not in selected_variables:
            selected_variables.append('1年以下')
        if '1-3年' not in selected_variables:
            selected_variables.append('1-3年')

        # 仅使用选定的变量进行 beta 计算
        X_selected = X_df[selected_variables].values

        # 计算 beta
        coefficients = calc_beta(X_selected, y)

        # 将 coefficients 扩展为 6 个值，未选中的变量系数设为 0
        full_coefficients = np.zeros(6)  # 初始化为 0
        for idx, col in enumerate(['1年以下', '1-3年', '3-5年', '5-7年', '7-10年', '10年以上']):
            if col in selected_variables:
                full_coefficients[idx] = coefficients[selected_variables.index(col)]

        # 将结果赋值到对应行
        df.loc[df['日期'] == date, beta_columns] = full_coefficients

    # 保存结果
    df.to_csv(output_dist + file_name, index=False)

reading data, loading into cache


100%|██████████| 1850/1850 [00:01<00:00, 949.11it/s] 


In [5]:
exclude_list = [
    '022015.OF.csv',  # 这个文件内只有一条数据，丢弃
    '022089.OF.csv',  # 这个文件内只有一条数据，丢弃
]  # 除了一半卡死了，记录会引起异常的文件
exclude_count = 0  # 记录已经处理过的文件的数量

for file_name in tqdm(cache.keys()):
    if file_name in exclude_list:  # 跳过不需要处理的文件
        continue

    if exclude_count < 0:  # 跳过已经处理过的文件
        exclude_count += 1
        continue

    process(file_name)

 37%|███▋      | 683/1850 [1:29:15<2:40:16,  8.24s/it]

empty dataframe: 021794.OF.csv


 84%|████████▍ | 1556/1850 [3:22:45<50:58, 10.40s/it]  

empty dataframe: 022014.OF.csv


 90%|████████▉ | 1664/1850 [3:36:28<27:56,  9.01s/it]

empty dataframe: 003868.OF.csv


100%|█████████▉| 1846/1850 [3:57:21<00:36,  9.11s/it]

empty dataframe: 003572.OF.csv


100%|██████████| 1850/1850 [3:57:29<00:00,  7.70s/it]


# 计算基金高频久期

读取文件 `债券久期.xlsx`，重命名后保存到 `processed_indexes/分段久期指数.csv`

In [6]:
import pandas as pd

# 读取数据
df = pd.read_excel('data/raw/债券久期.xlsx', header=1)

dest_df = pd.DataFrame()
dest_df['日期'] = df.iloc[:, 0]
dest_df['1年以下'] = df['中债国开行债券总指数(1年以下)财富指数']
dest_df['1-3年'] = df['中债-国开行债券总财富(1-3年)指数']
dest_df['3-5年'] = df['中债-国开行债券总财富(3-5年)指数']
dest_df['5-7年'] = df['中债-国开行债券总财富(5-7年)指数']
dest_df['7-10年'] = df['中债-国开行债券总财富(7-10年)指数']
dest_df['10年以上'] = df['中债国开行债券总指数(10年以上)财富指数']

dest_df.to_csv('data/processed_indexes/分段久期指数.csv', index=False)

计算基金久期

In [7]:
from collections import defaultdict
import pandas as pd
import os
from tqdm import tqdm

duration_index = pd.read_csv('data/processed_indexes/分段久期指数.csv')


result = pd.DataFrame()
cache = {}
# 读取数据
for file_name in os.listdir('data/eda_beta/'):
    df = pd.read_csv('data/eda_beta/' + file_name)
    code = file_name[:-4]
    cache[file_name] = df

duration_index.set_index('日期', inplace=True)
for df in cache.values():
    df.set_index('日期', inplace=True)

data = defaultdict(dict)
for date in tqdm(pd.date_range('2010-01-04', '2024-11-27')):
    date_str = date.strftime('%Y-%m-%d')
    for code, df in cache.items():
        if date_str in duration_index.index and date_str in df.index:
            duration_row = duration_index.loc[date_str]
            df_row = df.loc[date_str]
            
            duration = (
                duration_row['1年以下'] * df_row['beta_1年以下'] +
                duration_row['1-3年'] * df_row['beta_1-3年'] +
                duration_row['3-5年'] * df_row['beta_3-5年'] +
                duration_row['5-7年'] * df_row['beta_5-7年'] +
                duration_row['7-10年'] * df_row['beta_7-10年'] +
                duration_row['10年以上'] * df_row['beta_10年以上']
            )
            # 将结果存储到 result DataFrame 中
            data[date_str][code] = duration

result = pd.DataFrame.from_dict(data, orient='index')
result = result.fillna(0)
result.to_csv('data/eda_duration.csv')

100%|██████████| 5442/5442 [00:49<00:00, 110.43it/s]
